# Imports

In [14]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [15]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

---

# Armado de ventanas

In [16]:
for ventana_nro in range(1,6):
    installs["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= installs["created"].dt.day) & ((installs["created"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [17]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = installs.loc[installs["ventana_{}".format(ventana_nro)]]["ref_hash"].drop_duplicates().to_frame().set_index("ref_hash")

---

## Función generadora de features

In [18]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

### Cantidad de instalaciones por dia

In [19]:
installs['dia'] = installs['created'].dt.day


In [20]:
def cantidad_instalaciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["instalaciones_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [21]:
generar_feature_en_ventanas(installs, cantidad_instalaciones_por_dia, Xs)

---

# Guardado de los sets de entrenamiento

In [23]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/installs_v_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/intalls_v_{}.csv".format(ventana_nro))